<div style="text-align: center; margin: 50px">

<h1 style="color: white; background-color: grey; text-align: center;">Quantum Key distribution</h1>
<h3>Lab notebook</h3>

</div>

In [1]:
import numpy as np
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

from random import getrandbits
print('Libraries imported successfully!')

Libraries imported successfully!


## Coding cheat sheet:

#Defining a quantum circuit: 

`qc = QuantumCircuit(1)` #Define a 1 qubit quantum circuit <br>

`qc.x(0)` #Add an X gate <br>
`qc.h(0)` #Add an H gate <br>
`qc.z(0)` #Add a Z gate <br>
`qc.y(0)` #Add a Y gate <br>

`qc.draw()` #Draw the circuit <br>

**Using the statevector simulator** (Do this no matter which way you want to see the output):<br>

`svsim = Aer.get_backend('statevector_simulator')` # Tell it which simulator you want to use <br>
`job = execute(qc,svsim)` # Put in the name of your quantum circuit where it says qc<br>
`result = job.result()` <br>


See the output on the Bloch sphere:<br>
`state = result.get_statevector()` <br>
`plot_bloch_multivector(state)`<br>

See the output in vector form:<br>
`state = result.get_statevector()` <br>
`array_to_latex(state, pretext="\\text{Statevector} = ")` <br>

See the output in histogram form: <br>
`counts = result.get_counts(qc)` <br>
`plot_histogram(counts)` <br>

**Using the qasm simulator:**

`qc.measure_all()` #adds measurements <br>

`svsim = Aer.get_backend('qasm_simulator')` # Change statevector to qasm <br>
`job = execute(qc,svsim,shots=100)` # add shots - tell it how many times to run <br>
`result = job.result()` <br>


**Using a real quantum computer:**

Find the least busy backend: <br>
`IBMQ.load_account()` <br>
`provider = IBMQ.get_provider(hub='ibm-q')` <br>
`backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 2 
                                       and not x.configuration().simulator 
                                       and x.status().operational==True))` <br>
`print("least busy backend: ", backend)` <br>


Run the job:
`job = execute(qc, backend=backend, shots=100)`

`result = job.result()` <br>
`counts = result.get_counts(qc)` <br>
`plot_histogram(counts)` 





<a id="step1"></a>
# Coding the QKD protocol

Now that we have introduced you to the basic ideas behind QKD, let's try to code the actual protocol! In this notebook, we will give you the steps to code the protocol as well as some skeleton code. We will describe what you need to do at each step of the protocol. Try your best to complete the code to implement the protocol! It is completely okay (and expected) if you don't understand the details of each step. By the end of this session you will have a complete implementation, and you can play around with it on your own :) 


### The QKD protocol we will implement has the following steps:
1. The sender (Alice) generates a list of random bits (0s and 1s)
2. Alice generates a list of random encoding bases (X basis or Z basis)
3. Alice uses the lists of bits and bases to encode qubits using an encoding/decoding scheme
4. The receiver (Bob) receives Alice's qubits
5. Bob generates a list of random decoding bases (X basis or Z basis)
6. Bob makes measurements on the qubits using the decoding bases
7. Bob decodes the results of the measurements using the same encoding/decoding scheme
8. Alice and Bob compare their list of bases
9. Alice and Bob compare some of their bits to check for an eavesdropper. Based on the result, they either construct their key or find the eavesdropper.


The three major QKD ideas we introduced - bases, encoding/decoding, and comparison, will be relevant in these steps. Specifically, Alice will generate a list of bases in Step 2 and use them in Step 3, and Bob will generate a list of bases in Step 5 and use them in Step 6. Alice will encode qubits using the table we showed you in Step 3, and Bob will decode qubits using this table in Step 7. Alice and Bob will compare their bases and bits in Steps 8 and 9.

## Step 1: Alice generates a list of random bits (0s and 1s)

In this first step, Alice generates a list of random bits. Remember that the key is just a series of bits - what is quantum about QKD is not the key itself, but the way the key is shared between Alice and Bob. Some of the bits that Alice generates in this step will ultimately make it to the key.

There are many ways to generate random bits in Python. We will use the function `getrandbits` in the library `random`. We have already imported this function with the rest of our libraries. 

In the following block, we will use this function to generate a list of 500 random bits called `alice_bits`. 

In [2]:
# BLOCK 1 - Generate Alice's bits.

alice_bits = [] #This list will store Alice's bits
    
for i in range(500):   # Generate 500 random bits
    alice_bits.append(str(getrandbits(1))) # The function getrandbits generates 1 random bit

# Insert a print statement to see Alice's bits
print(alice_bits)

['0', '0', '1', '0', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '1', '1', '0', '1', '1', '0', '1', '1', '1', '1', '0', '1', '0', '0', '1', '0', '1', '1', '0', '1', '0', '1', '0', '1', '1', '0', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '1', '0', '0', '1', '0', '0', '1', '1', '1', '0', '0', '0', '0', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '1', '1', '1', '0', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '0', '0', '1', '0', '1', '0', '0', '0', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '0', '0', '1', '1', '0', '1', '1', '1', '0', '0', '0', '1', '0', '0', '1', '1', '1', '0', '0', '1', '1', '1', '1', '1', '0', '0', '0', '1', '0',

## Step 2 - Alice randomly chooses encoding bases

In this step, Alice will generate a random list of encoding bases. These encoding bases will be used to encode the bits from Step 1 into qubits. 

### What is a basis?
You can think of a basis as a set of two quantum states that lie along a common axis on the Bloch sphere, as we just discussed. IN QKD, we conventionally use two bases - X and Z. Do you remember which two states lie along the X basis, and which lie along the Z basis?

The |0> and |1> states lie along the Z axis! So, the Z basis consists of the states |0> and |1>. The |+> and |-> states lie along the X axis. So, the X basis consists of the states |+> and |->.

We will study how these bases are used in the next step of the protocol.

In our protocol, we will use these two encoding bases - the X basis and the Z basis. So, the four possible qubit states we will work with will be |0>, |1>, |+>, and |->.

In the next block, we have code to generate a list of random bases - X or Z. We will use the same function `getrandbits` to generate this list of bases.

In [3]:
# BLOCK 2 - Generate Alice's bases.

alice_bases = [] # List to store Alice's bases
for i in range(500):
    base = getrandbits(1)
    if base == 0:
        alice_bases.append("Z")
    else:
        #WRITE CODE HERE: Write code to append "X" to alice_bases
        alice_bases.append("X")

# WRITE CODE HERE: Add a print statement to display Alice's bases
print(alice_bases)

['Z', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'Z', 'Z',

<a id="step3"></a>
## Step 3 - Encode the classical bits into qubits using encoding bases


In this step, Alice will use the lists of bits and encodinf bases to qubits to send to Bob. The table below summarizes the qubit states Alice sends, based on the bit of Alice's `alice_bits` the corresponding basis of `alice_bases`:

| Bit in `alice_bits` | Corresponding basis in `alice_bases` | Encoding basis | Qubit state sent |
|:----------------:|:--------------------------:|:--------------------------:|:---------------:|
| 0 | Z | $$|0\rangle,|1\rangle$$ |$$|0\rangle$$ |
| 1 | Z | $$|0\rangle,|1\rangle$$ |$$|1\rangle$$ |
| 0 | X | $$|+\rangle,|-\rangle$$ |$$|+\rangle$$ |
| 1 | X | $$|+\rangle,|-\rangle$$ |$$|-\rangle$$ |

So, for example, if the first bit in `alice_bits` is 0 and the first basis in `alice_bases` is X, Alice will encode a |+> state qubit. If the second bit in `alice_bits` is 1 and the second basis in `alice_bases` is Z, Alice will encode a |1> state qubit.

In the next block, we will implement this encoding step using a for loop and conditional statements. Fill in the missing code to complete the encoding step!

In [4]:
#BLOCK 4 - Encode Alice's qubits. Remeber that the qubit will be in the |0> state at the start, before any gates are applied.

encoded_qubits = []
for i in range(500):
    qc = QuantumCircuit(1,1) #Creating a new quantum circuit for each bit-basis combination
    if alice_bases[i] == "Z":
        if alice_bits[i] == '0':
            #WRITE CODE HERE: What gate should we apply in this case?
            pass
        elif alice_bits[i] == '1':
            qc.x(0)
             #WRITE CODE HERE: What gate should we apply in this case?
    elif alice_bases[i] == "X":# WRITE CODE HERE: Complete the code for this elif condition to test if the current basis is X
        if alice_bits[i] == '0':# WRITE CODE HERE: Complete the code for this if condition to test if the current bit is 0
            qc.h(0) #WRITE CODE HERE: What gate should we apply in this case?
        elif alice_bits[i] == '1': # WRITE CODE HERE: Complete the code for this elif condition to test if the current bit is 1
            qc.x(0)
            qc.h(0) #WRITE CODE HERE: What gates should we apply in this case?
            
    encoded_qubits.append(qc) # Adding the qubit with the right state to the list of qubits that Alice will send Bob

# Step 4 - Alice sends qubits to Bob

### There's nothing we need to code here. Alice would send the qubits to Bob (maybe through a fiber optic cable)

## Step 5 - Bob randomly picks the bases he will use to measure Alices's qubits

Bob has received Alice's qubits and will now measure them. Before Bob measures these qubits, he needs to randomly generate a list of decoding bases. He will use these decoding bases while making measurements on the qubits and decoding them into classical bits.

In the block below, write code to generate a list of 500 bases. We have created an empty list for you to start with. 

Hint: You can copy the code used in Block 2 by Alice!

In [5]:
#BLOCK 4  - generate Bob's bases

bob_bases = []
# WRITE CODE HERE: Copy code from Block 2 to create the list of Bob's bases

# WRITE CODE HERE: Add a print statement to see Bob's bases
for i in range(500):
    base = getrandbits(1)
    if base == 0:
        bob_bases.append("Z")
    else:
        #WRITE CODE HERE: Write code to append "X" to alice_bases
        bob_bases.append("X")

# WRITE CODE HERE: Add a print statement to display Alice's bases
print(bob_bases)

['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'Z', 'X', 'X', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'X', 'X', 'X', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'X', 'Z', 'Z', 'Z', 'X', 'Z',

## Step 6 & 7 - Bob makes measurements and decodes qubits into bits

Bob now has to measure the qubits in the randomly generated list of decoding bases that he chose in Step 5. We have completed this step for you.

### Why is Bob decoding?
Why does Bob need to measureand decode qubits to bits? Remeber that the key that Alice and Bob share is ultimately a series of bits. Alice has a list of bits, and, by decoding Alice's qubits, Bob will generate a list of bits as well. Alice and Bob will use their lists of bits in later steps to create their key!

### Why does Bob need to select decoding bases?
Bob does not know which bases Alice used to encode the qubits, so the best that Bob can do it randomly guess. If Bob guesses correctly, Bob's decoded classical bit will be the same as Alice's bit. If Bob guesses incorrectly, Bob's decoded bit might be different from Alice's bit.

In [6]:
#BLOCK 6 - measure Alice's qubits. Remeber that if a bit bob_bases is 0, Bob measures in the Z basis. If the bit is 1,
# Bob measures in the X basis.

bob_bits = [] # List of Bob's bits generated from the results of Bob's measurements
    
for i in range(500):
    qc = encoded_qubits[i]
        
    if bob_bases[i] == "Z": # Bob's basis is Z
        qc.measure(0,0)
        

    elif bob_bases[i] == "X": # Bob's basis is X
        qc.h(0)
        qc.measure(0,0) # This is how an X basis measurement is made
            
            
      # Now that the measurements have been added to the circuit, let's run them.
    job = execute(qc, backend=Aer.get_backend('qasm_simulator'), shots = 1) 
    results = job.result()
    counts = results.get_counts()
    measured_bit = max(counts, key=counts.get)

        # Append measured bit to Bob's measured bitstring
    bob_bits.append(measured_bit) 
print(bob_bits)

['0', '0', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '0', '1', '1', '0', '1', '1', '1', '1', '0', '1', '0', '0', '1', '1', '1', '0', '1', '1', '0', '0', '1', '1', '1', '0', '1', '0', '0', '1', '0', '0', '1', '1', '1', '0', '0', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '0', '0', '1', '1', '0', '1', '0', '0', '0', '1', '0', '0', '1', '1', '1', '0', '1', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '1', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '1', '1', '0', '1', '0', '1', '1', '1', '1', '1', '1', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '1', '1', '1', '0', '0', '1', '1', '0', '1', '0', '1', '1', '0', '1', '0', '0', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '0', '1', '0', '1', '1', '1', '0', '0', '1', '1', '1', '0', '1', '0', '0', '0', '0', '1', '0', '0', '1', '1', '1', '0', '1', '0', '1', '1', '0', '1', '1', '0', '0', '0', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '0',

## Step 8 - Alice and Bob compare their bases

Alice and Bob now both have a list of bits, and a list of bases. Their next step is to compare their list of bases to see which of their bases match. 

Here is the punchline of the QKD protocol - **If their bases match, their bits have to match as well.** Think about this statement for a minute - why is it true? Try out an example yourself - what if Alice encodes a 0 on the X basis, and Bob also measures it in the X basis? Is Bob's bit guaranteed to be the same as Alice's? What if, instead, Bob had measured in the Z basis? Are the bits still guaranteed to be the same?

So, by throwing away the bits for which their bases do not match, they will be left with a matching series of bits! This will be their key.

In the next block, complete the code to compare Alice and Bob's bases and store the indices of the bases that match in the list `agreeing_indices`. 

Can you guess roughly how many of their bases should match? This will also be the length of their keys! Remember that they randomly picked one of two bases - X or Z.

In [7]:
#BLOCK 7 - Alice and Bob compare their bases

agreeing_indices = []
    
for i in range(500):
    if alice_bases[i] == bob_bases[i]:
        agreeing_indices.append(i) # WRITE CODE HERE. Add code to compare Alice and Bob's bases one-by-one. If their bases at a given index match, append that index to agreeing_indices
    
len(agreeing_indices) # This statement will print how many bases were the same for Alice and Bob. Can you guess what this number should approximately be?

253

## Step 9 - Alice and Bob generate their key

The step you've been waiting for! Alice and Bob now know which of their bases match, and can now generate their keys. In the next two blocks, Alice and Bob will create their keys using the list of matching indices `agreeing_indices`.

We have completed the block making Alice's key. Can you complete the block with Bob's key?

Next, Alice and Bob will compare the first 10 bits of their keys, to check if they are the same.

In [8]:
#BLOCK 8 - create Alice's key

alice_key = []
for index in agreeing_indices:
    alice_key.append(alice_bits[index])
    
# WRITE CODE HERE: Add a print statement to see Alice's key
print(alice_key)

['0', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '1', '0', '1', '1', '0', '1', '0', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '0', '1', '0', '1', '1', '0', '1', '1', '1', '1', '1', '0', '0', '1', '0', '1', '0', '0', '0', '1', '1', '1', '1', '0', '0', '1', '0', '0', '1', '0', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '0', '1', '1', '0', '0', '1', '1', '0', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '1', '1', '0', '0', '1', '0', '0', '1', '0', '1', '1', '0', '1', '1', '1', '1', '1', '0', '1', '1', '0', '0', '1', '1', '0', '1', '1', '0', '1', '1', '1', '1', '0', '1', '0', '0', '1', '0', '0', '1', '1', '1', '1', '1', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '1', '1', '0', '1', '0', '1', '1', '1', '1', '0', '1', '0', '0',

In [9]:
#BLOCK 9 - create Bob's key

bob_key = []

# WRITE CODE HERE: Add code to create Bob's key
# WRITE CODE HERE: Add a print staement to see Bob's key
for index in agreeing_indices:
    bob_key.append(bob_bits[index])
    
# WRITE CODE HERE: Add a print statement to see Alice's key
print(bob_key)

['0', '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '1', '0', '1', '1', '0', '1', '0', '1', '1', '1', '0', '0', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '0', '1', '0', '1', '1', '0', '1', '1', '1', '1', '1', '0', '0', '1', '0', '1', '0', '0', '0', '1', '1', '1', '1', '0', '0', '1', '0', '0', '1', '0', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '0', '1', '1', '0', '0', '1', '1', '0', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '1', '1', '0', '0', '1', '0', '0', '1', '0', '1', '1', '0', '1', '1', '1', '1', '1', '0', '1', '1', '0', '0', '1', '1', '0', '1', '1', '0', '1', '1', '1', '1', '0', '1', '0', '0', '1', '0', '0', '1', '1', '1', '1', '1', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '1', '1', '0', '1', '0', '1', '1', '1', '1', '0', '1', '0', '0',

In [10]:
#BLOCK 10 - we can see that these keys are the same

print(alice_key == bob_key)
# WRITE CODE HERE: Add code to compare the first 10 bits of Alice and Bob's keys to see if they are the same.

True


## The keys are the same! Alice and Bob have created a key.

## Optional - Encryping and decrypting messages

In [11]:
# BLOCK 11 - Functions to use the key to encrypt the secret message and decrypt the message

import binascii

def encrypt_message(unencrypted_string, key):
    # Convert ascii string to binary string
    bits = bin(int(binascii.hexlify(unencrypted_string.encode('utf-8', 'surrogatepass')), 16))[2:]
    bitstring = bits.zfill(8 * ((len(bits) + 7) // 8))
    # created the encrypted string using the key
    encrypted_string = ""
    for i in range(len(bitstring)):
        encrypted_string += str( (int(bitstring[i])^ int(key[i])) )
    return encrypted_string
    
def decrypt_message(encrypted_bits, key):
    # created the unencrypted string using the key
    unencrypted_bits = ""
    for i in range(len(encrypted_bits)):
        unencrypted_bits += str( (int(encrypted_bits[i])^ int(key[i])) )
    # Convert bitstring into
    i = int(unencrypted_bits, 2)
    hex_string = '%x' % i
    n = len(hex_string)
    bits = binascii.unhexlify(hex_string.zfill(n + (n & 1)))
    unencrypted_string = bits.decode('utf-8', 'surrogatepass')
    return unencrypted_string

In [12]:
# BLOCK 12 - Using the key to send and receive a secret message

message = "QKD is cool!"
print("Original Messge:", message)
# Call the function encrypt_message with the right inputs
encrypted_message = encrypt_message(message,alice_key)
print("Encrypted message:", encrypted_message)
# Call the function deencrypt_message with the right inputs
decrypted_message = decrypt_message(encrypted_message, bob_key)
print("Decrypted message:", decrypted_message)

Original Messge: QKD is cool!
Encrypted message: 000111101010100110010011000101111001110110110011000110101011110001000111100111010011101111011000
Decrypted message: QKD is cool!


# Optional homework
## Implementing an eavesdropper

Can you write code to implement the eavesdropper? Remember that the eavesdropper intercepts Alice's qubits and measures them.
Then, the eavesdropper sends the measured qubits to Bob.
So, the eavesdropper's code would be very similar to Bob's code.
We recommend re-implementing the whole protocol with an eavesdropper in a new notebook.
Does you code show that Alice and Bob's keys will not match if there is an eavesdropper? Post your code on Piazza!